In [8]:
# !pip install ipydatagrid
%connect_info

{
  "shell_port": 62427,
  "iopub_port": 62428,
  "stdin_port": 62429,
  "control_port": 62431,
  "hb_port": 62430,
  "ip": "127.0.0.1",
  "key": "da5304bb-4a5d226ece94bd1ffc936dbf",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-184f50ef-6e96-42c5-99ca-33c83e277269.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [1]:
import os
import sys
import logging
import json, logging.config
localPath = "c:\\Users\\djhalama\\Documents\\GitHub\\BUS"
if os.path.exists(localPath):
    # print(f"localPath exists: {localPath}")
    os.chdir("c:\\Users\\djhalama\\Documents\\GitHub\\BUS")
else:
    from google.colab import drive
    drive.mount('/content/gdrive')
    os.chdir('/content/gdrive/MyDrive/BUS_Project_Home/Share_with_group/David_Halama/BUS')
    sys.path.append('/content/gdrive/MyDrive/BUS_Project_Home/Share_with_group/David_Halama/BUS/Segmentation')
from Common import Common
# https://coralogix.com/log-analytics-blog/python-logging-best-practices-tips/
with open('logging-config.json', 'rt') as f:
    config = json.load(f)
    logging.config.dictConfig(config)

print(os.getcwd())

c:\Users\djhalama\Documents\GitHub\BUS


In [2]:
from ProcessImage import busUI

def run():
    logger = logging.getLogger("BUStest")
    print("inrun")
    # pimg.load(np.arange(1,144)) #80, 101, 125
    # pimg.display7((80,))
    # pimg.runSaveGTStats(np.arange(1, 144))
    # pimg.segList.saveROIStats()
    logger.error("help me")
    bus = busUI()
    bus.initUI()

run()

help me


inrun
